# Waterfilling Levels

In [1]:
import sys
sys.path.insert(1, '../../functions_multi_resource')
import importlib
import numpy as np
import nbformat
import plotly.express
import plotly.express as px
import pandas as pd
from scipy.optimize import minimize
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from food_bank_functions import *
import time
# importlib.reload(food_bank_functions)

In [2]:
np.random.seed(3435)

In [3]:
n = 3
k = 4
size = [1., 2., 3.]
B = np.zeros((k, n*k))
for i in range(n):
    B[:,k*i:k*(i+1)] = size[i]*np.eye(k)
print(B)

[[1. 0. 0. 0. 2. 0. 0. 0. 3. 0. 0. 0.]
 [0. 1. 0. 0. 0. 2. 0. 0. 0. 3. 0. 0.]
 [0. 0. 1. 0. 0. 0. 2. 0. 0. 0. 3. 0.]
 [0. 0. 0. 1. 0. 0. 0. 2. 0. 0. 0. 3.]]


# Generating Distribution

In [4]:
n = 6
k = 9

#### Different Population of Locations

In [5]:
# size = [1, 1, 1, 1, 1, 1]
size = [928, 1200, 420, 429, 103, 393]
size = size / np.sum(size) * 100
county = ['Broome', 'Steuben', 'Chemung', 'Tioga', 'Schuyler', 'Tompkins']
print(county)
print(size)

['Broome', 'Steuben', 'Chemung', 'Tioga', 'Schuyler', 'Tompkins']
[26.72041463 34.55226029 12.0932911  12.35243305  2.96573568 11.31586525]


#### Distribution on Weights

In [6]:
product = ['cereal', 'diapers', 'pasta', 'paper', 'prepared_meals', 'rice', 'meat', 'fruit', 'produce']
w = [3.9, 3.5, 3.2, 3, 2.8, 2.7, 1.9, 1.2, .2]
print(product)
print(w)
budget = np.sum(w)*np.ones(k)
print(budget)

['cereal', 'diapers', 'pasta', 'paper', 'prepared_meals', 'rice', 'meat', 'fruit', 'produce']
[3.9, 3.5, 3.2, 3, 2.8, 2.7, 1.9, 1.2, 0.2]
[22.4 22.4 22.4 22.4 22.4 22.4 22.4 22.4 22.4]


In [7]:
w_1 = [1, 0, 1, 0, 0, 1, 1, 1, 1] # soup kitchen 
w_2 = [1, 1, 1, 1, 1, 1, 1, 1, 1] # general warehouse
w_3 = np.random.randint(0,2,9)
w_4 = np.random.randint(0,2,9)
w_5 = np.random.randint(0,2,9)
w_6 = np.random.randint(0,2,9)
w_7 = np.random.randint(0,2,9)
w_8 = np.random.randint(0,2,9)
weight_matrix = np.asarray([ np.multiply(w_1,w), np.multiply(w_2,w), np.multiply(w_3,w), np.multiply(w_4,w), np.multiply(w_5,w), np.multiply(w_6,w), np.multiply(w_7,w), np.multiply(w_8,w)])
print(weight_matrix)
weight_distribution = [1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8]

[[3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]]


In [8]:
expected_weights = np.zeros((n,k))

In [9]:
for i in range(n):
    for j in range(k):
#         print(i,j)
        expected_weights[i,j] = w[j] * (1/8) * (w_1[j] + w_2[j] + w_3[j] + w_4[j] + w_5[j] + w_6[j] + w_7[j]+w_8[j])
print(expected_weights)

[[3.4125 1.75   2.     1.5    2.1    1.35   1.425  1.05   0.175 ]
 [3.4125 1.75   2.     1.5    2.1    1.35   1.425  1.05   0.175 ]
 [3.4125 1.75   2.     1.5    2.1    1.35   1.425  1.05   0.175 ]
 [3.4125 1.75   2.     1.5    2.1    1.35   1.425  1.05   0.175 ]
 [3.4125 1.75   2.     1.5    2.1    1.35   1.425  1.05   0.175 ]
 [3.4125 1.75   2.     1.5    2.1    1.35   1.425  1.05   0.175 ]]


In [10]:
size

array([26.72041463, 34.55226029, 12.0932911 , 12.35243305,  2.96573568,
       11.31586525])

In [11]:
x, sol = solve(expected_weights, n, k, budget, size)

In [12]:
x = np.reshape(x, (n,k))

In [13]:
print(x)

[[0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]]


In [14]:
print(excess(x, budget, size))
print(envy_utility(x, expected_weights))
print(proportionality_utility(x, expected_weights, size, budget))

[1.18423789e-15 1.18423789e-15 1.18423789e-15 1.18423789e-15
 1.18423789e-15 1.18423789e-15 1.18423789e-15 1.18423789e-15
 1.18423789e-15]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]


In [15]:
x.shape

(6, 9)

In [16]:
print(n)

6


### Test

In [17]:
obs_types = np.random.randint(0,8,n)
print(obs_types)

observed_weights = np.zeros((n, k))
for i in range(n):
    observed_weights[i,:] = weight_matrix[obs_types[i], :]
print(observed_weights)

[6 5 3 7 7 0]
[[3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]]


In [18]:
opt, _ = solve(observed_weights, n, k, budget, size)
opt = np.reshape(opt, (n,k))
print(np.around(opt, decimals=4))

[[0.2136 0.     0.8383 0.     0.     0.     0.1503 0.8383 0.0914]
 [0.4831 0.     0.     0.     0.6483 0.     0.532  0.     0.5776]
 [0.     0.     0.     1.8523 0.     0.     0.     0.     0.    ]
 [0.     1.4852 0.     0.     0.     0.0061 0.     0.     0.    ]
 [0.     1.3671 0.     0.     0.     0.159  0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.9312 0.     0.     0.    ]]


In [19]:
print(excess(opt, budget, size))
print(envy_utility(opt, observed_weights))
print(proportionality_utility(opt, observed_weights, size, budget))

[ 1.32634644e-13  6.90529115e-12  1.28489811e-11  4.00213196e-12
 -9.82799027e-12  4.91636361e-12 -8.99428680e-13  6.07987734e-12
  3.08493971e-13]
[-1.81524567e+00 -3.68854118e+00 -3.42365928e-01 -4.87440507e-05
  4.87440507e-05 -3.88666041e-01]
[-2.49605583 -2.85443149 -0.9648     -2.01123407 -2.01118533 -2.27992187]


In [20]:
allocation = et_full(expected_weights, observed_weights, n, k, budget, size)

../../functions_multi_resource\food_bank_functions.py:20: RuntimeWarning: divide by zero encountered in log
  value[i] = np.log(np.dot(X[i,:], W[i,:]))


In [21]:
budget_used = np.zeros(k)
for i in range(n):
    budget_used += size[i] * allocation[i,:]

In [22]:
budget

array([22.4, 22.4, 22.4, 22.4, 22.4, 22.4, 22.4, 22.4, 22.4])

In [23]:
budget - budget_used

array([ 8.79512546e+00, -3.55271368e-15,  0.00000000e+00,  3.88697642e+00,
        0.00000000e+00, -3.55271368e-15,  0.00000000e+00,  2.24000000e+01,
        0.00000000e+00])

In [24]:
allocation.shape

(6, 9)

In [25]:
np.sum(allocation, axis=0)

array([5.93216940e-01, 1.82895139e+00, 8.38310345e-01, 1.58169080e+00,
       6.48293333e-01, 1.83104078e+00, 8.38310345e-01, 1.84133364e-10,
       9.35967819e-01])

In [26]:
print(np.around(allocation, decimals=5))

[[0.      0.      0.83831 0.      0.      0.      0.83831 0.      0.     ]
 [0.29661 0.      0.      0.      0.64829 0.      0.      0.      0.5082 ]
 [0.      0.74091 0.      0.79085 0.      0.8405  0.      0.      0.     ]
 [0.      1.08805 0.      0.      0.      0.99054 0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.      0.      0.      0.     ]
 [0.29661 0.      0.      0.79085 0.      0.      0.      0.      0.42777]]


In [27]:
print(excess(allocation, budget, size))
print(envy_utility(allocation, observed_weights))
print(proportionality_utility(allocation, observed_weights, size, budget))

[ 1.46585424e+00 -5.92118946e-16  0.00000000e+00  6.47829404e-01
  0.00000000e+00 -5.92118946e-16  0.00000000e+00  3.73333333e+00
  0.00000000e+00]
[-3.01696991 -1.48084452 -0.75242919 -1.620107    6.48262349  3.03305598]
[-1.94578276 -1.10243418 -2.64305268 -3.27942349  3.2032      1.69207322]


In [28]:
print(np.max(allocation - opt))

0.984486982654082


In [29]:
observed_weights[3,:]

array([3.9, 3.5, 0. , 0. , 2.8, 2.7, 0. , 1.2, 0.2])

In [30]:
allocation = et_online(expected_weights, observed_weights, n, k, budget, size)

c:\users\seanr\anaconda3\envs\food-bank\lib\site-packages\scipy\optimize\_constraints.py:422: OptimizeWarning: Equality and inequality constraints are specified in the same element of the constraint list. For efficient use with this method, equality and inequality constraints should be specified in separate elements of the constraint list. 
  "in separate elements of the constraint list. ", OptimizeWarning)


In [31]:
print(np.around(allocation, decimals=5))

[[0.      0.      0.83831 0.      0.      0.      0.83831 0.      0.     ]
 [0.54946 0.      0.      0.      0.64829 0.      0.      0.      0.62438]
 [0.      0.80208 0.      0.77013 0.      0.75096 0.      0.      0.     ]
 [0.      1.02815 0.      0.      0.      1.07821 0.      0.      0.     ]
 [0.57924 0.      0.      0.      0.      0.      0.      1.76656 0.09087]
 [0.14997 0.      0.      0.77013 0.      0.      0.      1.51653 0.0492 ]]


In [32]:
print(excess(allocation, budget, size))
print(envy_utility(allocation, observed_weights))
print(proportionality_utility(allocation, observed_weights, size, budget))

[ 0.00000000e+00 -5.92118946e-16  0.00000000e+00  7.28657279e-01
  0.00000000e+00 -5.92118946e-16  0.00000000e+00  5.92118946e-16
  0.00000000e+00]
[ 0.12169717 -1.80578398 -0.63556618 -1.67481904  2.11260958  1.98253112]
[-1.94578276 -2.11179524 -2.5532557  -3.30648951 -1.19387993  0.51985119]


In [33]:
alloc = hope_online(weight_matrix, np.asarray(weight_distribution), obs_types, n, k, budget, size)

In [34]:
print(np.around(alloc, decimals=10))

[[0.3979273  0.         0.52733154 0.         0.         0.
  0.51328071 0.5779406  0.54704707]
 [0.29871208 0.         0.         0.         0.5686264  0.
  0.22046779 0.         0.19756405]
 [0.         0.83234721 0.10295311 0.62915128 0.         0.6132469
  0.         0.01908847 0.        ]
 [0.03854269 0.8724362  0.         0.         0.04320113 1.00638528
  0.         0.41393244 0.01785953]
 [0.20554075 0.35556972 0.         0.         0.17266822 0.58273797
  0.         0.36830864 0.09890404]
 [0.03184594 0.04444622 0.52733154 0.62915128 0.1508453  0.07284225
  0.0943159  0.04603858 0.03910077]]


In [35]:
print(excess(allocation, budget, size))
print(envy_utility(allocation, observed_weights))
print(proportionality_utility(allocation, observed_weights, size, budget))

[ 0.00000000e+00 -5.92118946e-16  0.00000000e+00  7.28657279e-01
  0.00000000e+00 -5.92118946e-16  0.00000000e+00  5.92118946e-16
  0.00000000e+00]
[ 0.12169717 -1.80578398 -0.63556618 -1.67481904  2.11260958  1.98253112]
[-1.94578276 -2.11179524 -2.5532557  -3.30648951 -1.19387993  0.51985119]


In [36]:
alloc = hope_full(weight_matrix, np.asarray(weight_distribution), obs_types, n, k, budget, size)

In [37]:
print(np.around(alloc, decimals=3))

[[0.398 0.    0.527 0.    0.    0.    0.513 0.578 0.547]
 [0.341 0.    0.    0.    0.569 0.    0.251 0.    0.225]
 [0.    0.814 0.    0.647 0.    0.606 0.    0.    0.   ]
 [0.    0.968 0.    0.    0.    0.916 0.    0.    0.   ]
 [0.    0.2   0.    0.    0.    0.886 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.1   0.    0.    0.   ]]


In [38]:
print(excess(allocation, budget, size))
print(envy_utility(allocation, observed_weights))
print(proportionality_utility(allocation, observed_weights, size, budget))

[ 0.00000000e+00 -5.92118946e-16  0.00000000e+00  7.28657279e-01
  0.00000000e+00 -5.92118946e-16  0.00000000e+00  5.92118946e-16
  0.00000000e+00]
[ 0.12169717 -1.80578398 -0.63556618 -1.67481904  2.11260958  1.98253112]
[-1.94578276 -2.11179524 -2.5532557  -3.30648951 -1.19387993  0.51985119]


In [39]:
allocation = hope_full_v2(weight_matrix, np.asarray(weight_distribution), obs_types, n, k, budget, size)

../../functions_multi_resource\food_bank_functions.py:63: RuntimeWarning: divide by zero encountered in log
  value[i] = np.sum([weight_distribution[i,j] * np.log(np.dot(X[i,:], weight_matrix[j,:])) for j in range(num_types)])
../../functions_multi_resource\food_bank_functions.py:63: RuntimeWarning: invalid value encountered in double_scalars
  value[i] = np.sum([weight_distribution[i,j] * np.log(np.dot(X[i,:], weight_matrix[j,:])) for j in range(num_types)])


In [40]:
print(np.around(allocation, decimals=5))

[[0.      0.      0.83831 0.      0.      0.      0.47818 0.156   0.     ]
 [0.14319 0.      0.      0.      0.64829 0.      0.2785  0.      0.44152]
 [0.      1.15894 0.      0.47027 0.      1.21395 0.      0.      0.     ]
 [0.      0.67879 0.      0.      0.      0.62492 0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.      0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.      0.      0.      0.     ]]


In [41]:
print(excess(alloc, budget, size))
print(envy_utility(alloc, observed_weights))
print(proportionality_utility(alloc, observed_weights, size, budget))

[ 0.00000000e+00 -5.92118946e-16  1.38491375e+00  2.42984711e+00
  4.58778742e-01 -5.92118946e-16 -5.92118946e-16  1.15953127e+00
  0.00000000e+00]
[-3.16672803 -0.80641556 -0.56445842 -1.37568772  2.76943467  4.74785425]
[-2.68794889 -1.47177479 -1.83417378 -2.65851536  0.11091932  2.66470536]


In [42]:
allocation = hope_online_v2(weight_matrix, np.asarray(weight_distribution), obs_types, n, k, budget, np.asarray(size))

In [43]:
print(np.around(allocation, decimals=5))

[[0.      0.      0.83831 0.      0.      0.      0.47818 0.156   0.     ]
 [0.2814  0.      0.      0.      0.64829 0.      0.2785  0.      0.53202]
 [0.      1.83493 0.      1.85227 0.      1.85227 0.      0.      0.     ]
 [0.      0.01697 0.      0.      0.      0.      0.      0.17946 0.     ]
 [0.      0.      0.      0.      0.      0.      0.      0.      0.     ]
 [1.1203  0.      0.      0.      0.      0.      0.      1.41524 0.35503]]


In [44]:
print(excess(alloc, budget, size))
print(envy_utility(alloc, observed_weights))
print(proportionality_utility(alloc, observed_weights, size, budget))

[ 0.00000000e+00 -5.92118946e-16  1.38491375e+00  2.42984711e+00
  4.58778742e-01 -5.92118946e-16 -5.92118946e-16  1.15953127e+00
  0.00000000e+00]
[-3.16672803 -0.80641556 -0.56445842 -1.37568772  2.76943467  4.74785425]
[-2.68794889 -1.47177479 -1.83417378 -2.65851536  0.11091932  2.66470536]


## Experiment

In [45]:
group = np.arange(n)
num_iterations = 1000

budget = [np.sum(w) for j in range(k)]


# 8 different algorithms
run_time = np.zeros((5, num_iterations))
env = np.zeros((8,num_iterations))
po = np.zeros((8,num_iterations))
prop = np.zeros((8,num_iterations))
linf = np.zeros((8,num_iterations))
l1 = np.zeros((8, num_iterations))
max_min = np.zeros((8, num_iterations))

for i in range(num_iterations):
    print(i)
    obs_types = np.random.randint(0,8,n)
    print(obs_types)

    observed_weights = np.zeros((n, k))
    for i in range(n):
        observed_weights[i,:] = weight_matrix[obs_types[i], :]
    print(observed_weights)

    start = time.perf_counter()
    opt, _ = solve(observed_weights, n, k, budget, size)
    run_time[0,i] = time.perf_counter() - start
    
    opt = np.reshape(opt, (n,k))
    
    start = time.perf_counter()
    et_full_alloc = et_full(expected_weights, observed_weights, n, k, budget, size)
    run_time[1,i] = time.perf_counter() - start
    
    start = time.perf_counter()
    et_online_alloc = et_online(expected_weights, observed_weights, n, k, budget, size)
    run_time[2,i] = time.perf_counter() - start
    
    prop_alloc = proportional_alloc(n, k, np.asarray(budget), size)
    
    start = time.perf_counter()
    weight_full_alloc = hope_full(weight_matrix, np.asarray(weight_distribution), obs_types, n, k, budget, size)
    run_time[3,i] = time.perf_counter() - start
    
    start = time.perf_counter()
    hope_online_alloc = hope_online(weight_matrix, np.asarray(weight_distribution), obs_types, n, k, budget, size)
    run_time[4,i] = time.perf_counter() - start
    
    
    hope_full_v2_alloc = hope_full_v2(weight_matrix, np.asarray(weight_distribution), obs_types, n, k, budget, size)
    hope_online_v2_alloc = hope_online_v2(weight_matrix, np.asarray(weight_distribution), obs_types, n, k, budget, size)
    offline = offline_alloc(weight_matrix, weight_distribution, n, k, budget, size)
    
    # comparing proportional_allocation
    env[0,i] = np.max(envy_utility(prop_alloc, observed_weights))
    po[0,i] = np.max(excess(prop_alloc, budget, size))
    prop[0,i] = np.amax(proportionality_utility(prop_alloc, observed_weights, size, budget))
    linf[0,i] = np.amax(np.abs(opt - prop_alloc))    
    l1[0,i] = np.sum(np.abs(opt - prop_alloc)) 
    max_min[0,i] = np.max(np.abs(utility(prop_alloc, observed_weights) - utility(opt, observed_weights)))
    
    # comparing et_online

    env[1,i] = np.max(envy_utility(et_online_alloc, observed_weights))
    po[1,i] = np.max(excess(et_online_alloc, budget, size))
    prop[1,i] = np.amax(proportionality_utility(et_online_alloc, observed_weights, size, budget))
    linf[1,i] = np.amax(np.abs(opt - et_online_alloc))
    l1[1,i] = np.sum(np.abs(opt - et_online_alloc))
    max_min[1,i] = np.max(np.abs(utility(et_online_alloc, observed_weights) - utility(opt, observed_weights)))  
    
    # comparing et_full

    env[2,i] = np.max(envy_utility(et_full_alloc, observed_weights))
    po[2,i] = np.max(excess(et_full_alloc, budget, size))
    prop[2,i] = np.amax(proportionality_utility(et_full_alloc, observed_weights, size, budget))
    linf[2,i] = np.amax(np.abs(opt - et_full_alloc))    
    l1[2,i] = np.sum(np.abs(opt - et_full_alloc))
    max_min[2,i] = np.max(np.abs(utility(et_full_alloc, observed_weights) - utility(opt, observed_weights)))
    
    
    # comparing hope_online

    env[3,i] = np.max(envy_utility(hope_online_alloc, observed_weights))
    po[3,i] = np.max(excess(hope_online_alloc, budget, size))
    prop[3,i] = np.amax(proportionality_utility(hope_online_alloc, observed_weights, size, budget))
    linf[3,i] = np.amax(np.abs(opt - hope_online_alloc)) 
    l1[3,i] = np.sum(np.abs(opt - hope_online_alloc))
    max_min[3,i] = np.max(np.abs(utility(hope_online_alloc, observed_weights) - utility(opt, observed_weights)))
    
    
    # comparing hope_full
    

    env[4,i] = np.max(envy_utility(weight_full_alloc, observed_weights))
    po[4,i] = np.max(excess(weight_full_alloc, budget, size))
    prop[4,i] = np.amax(proportionality_utility(weight_full_alloc, observed_weights, size, budget))
    linf[4,i] = np.amax(np.abs(opt - weight_full_alloc))
    l1[4, i] = np.sum(np.abs(opt - weight_full_alloc))
    max_min[4,i] = np.max(np.abs(utility(weight_full_alloc, observed_weights) - utility(opt, observed_weights)))
    
    
    # comparing hope_online_v2
    env[5,i] = np.max(envy_utility(hope_online_v2_alloc, observed_weights))
    po[5,i] = np.max(excess(hope_online_v2_alloc, budget, size))
    prop[5,i] = np.amax(proportionality_utility(hope_online_v2_alloc, observed_weights, size, budget))
    linf[5,i] = np.amax(np.abs(opt - hope_online_v2_alloc)) 
    l1[5, i] = np.sum(np.abs(opt - hope_online_v2_alloc))
    max_min[5,i] = np.max(np.abs(utility(hope_online_v2_alloc, observed_weights) - utility(opt, observed_weights)))
    
    
    # comparing hope_full_v2
    env[6,i] = np.max(envy_utility(hope_full_v2_alloc, observed_weights))
    po[6,i] = np.max(excess(hope_full_v2_alloc, budget, size))
    prop[6,i] = np.amax(proportionality_utility(hope_full_v2_alloc, observed_weights, size, budget))
    linf[6,i] = np.amax(np.abs(opt - hope_full_v2_alloc)) 
    l1[6,i] = np.sum(np.abs(opt - hope_full_v2_alloc))
    max_min[6,i] = np.max(np.abs(utility(hope_full_v2_alloc, observed_weights) - utility(opt, observed_weights)))
    
    # comparing offline
    env[7,i] = np.max(envy_utility(offline, observed_weights))
    po[7,i] = np.max(excess(offline, budget, size))
    prop[7,i] = np.amax(proportionality_utility(offline, observed_weights, size, budget))
    linf[7,i] = np.amax(np.abs(opt - offline))
    l1[7,i] = np.sum(np.abs(opt - offline)) 
    max_min[7,i] = np.max(np.abs(utility(offline, observed_weights) - utility(opt, observed_weights)))
    
    

0
[7 2 3 5 5 3]
[[3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]]
1
[2 0 4 2 1 6]
[[0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]]
2
[0 2 2 4 2 1]
[[3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]]
3
[5 5 7 1 0 3]
[[3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.

33
[3 5 6 2 0 0]
[[3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]]
34
[1 7 4 7 5 1]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]]
35
[1 4 0 0 7 7]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]]
36
[4 0 2 7 1 1]
[[3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.

66
[7 2 5 6 0 1]
[[3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]]
67
[6 2 7 7 4 7]
[[3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]]
68
[4 1 1 6 0 1]
[[3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]]
69
[1 0 3 0 5 3]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.

99
[6 1 3 0 4 5]
[[3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]]
100
[1 6 3 5 2 1]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]]
101
[1 6 5 1 6 1]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]]
102
[4 4 1 2 4 4]
[[3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9

132
[5 5 3 3 5 1]
[[3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]]
133
[4 5 7 5 3 2]
[[3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]]
134
[4 7 0 2 5 7]
[[3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]]
135
[2 2 2 1 7 6]
[[0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.

...

859
[4 0 6 3 3 3]
[[3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]]
860
[1 3 7 0 4 5]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]]
861
[3 7 5 4 4 1]
[[3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]]
862
[1 1 0 1 4 0]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.

892
[2 1 5 1 5 7]
[[0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]]
893
[1 4 1 1 0 6]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]]
894
[7 1 6 5 5 2]
[[3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]]
895
[5 0 7 7 7 6]
[[3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.

925
[4 4 1 6 5 0]
[[3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]]
926
[7 2 6 3 5 3]
[[3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]]
927
[7 0 6 4 0 0]
[[3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]]
928
[3 4 5 0 2 3]
[[3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 3.2 3.  2.8 2.7 0.

958
[0 5 3 5 0 4]
[[3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]]
959
[1 2 7 3 7 2]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]]
960
[1 2 3 0 5 2]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]]
961
[2 5 7 4 3 6]
[[0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.

991
[2 2 0 2 4 5]
[[0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]]
992
[5 7 5 2 0 0]
[[3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  0.  2.8 0.  1.9 0.  0.2]
 [0.  3.5 3.2 3.  2.8 0.  1.9 1.2 0. ]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]]
993
[4 6 3 1 3 4]
[[3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]]
994
[3 6 7 4 3 1]
[[3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  3.2 0.  0.  0.  1.9 1.2 0.2]
 [3.9 3.5 0.  0.  2.8 2.7 0.  1.2 0.2]
 [3.9 0.  0.  3.  2.8 0.  1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 0.  1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.

In [46]:
env_std = np.std(env, axis=1) * 1.96 / np.sqrt(num_iterations)
po_std = np.std(po, axis=1) * 1.96 / np.sqrt(num_iterations)
prop_std = np.std(prop, axis=1) * 1.96 / np.sqrt(num_iterations)
linf_std = np.std(linf, axis=1) * 1.96 / np.sqrt(num_iterations)
l1_std = np.std(l1, axis=1) * 1.96 / np.sqrt(num_iterations)
max_min_std = np.std(max_min, axis=1) * 1.96 / np.sqrt(num_iterations)

In [47]:
print(1.96*env_std / np.sqrt(num_iterations))

[0.         0.00019623 0.00030249 0.00015057 0.00024952 0.00193414
 0.00035249 0.        ]


In [48]:
print(1.96*po_std / np.sqrt(num_iterations))

[1.43791707e-19 8.59388020e-05 2.19970993e-04 3.99732964e-05
 2.08954098e-04 7.18958536e-20 4.27917045e-04 1.43791707e-19]


In [49]:
print(1.96*prop_std / np.sqrt(num_iterations))

[0.00000000e+00 1.30763260e-04 2.88699158e-04 9.88889529e-05
 9.63151547e-05 6.09243527e-04 2.62073297e-04 0.00000000e+00]


In [50]:
print(1.96*linf_std / np.sqrt(num_iterations))

[4.79705097e-05 1.57106026e-04 7.51688815e-05 1.19489246e-04
 6.82611557e-05 5.46915750e-04 8.06451278e-05 4.79705097e-05]


In [51]:
print(1.96*l1_std / np.sqrt(num_iterations))

[0.0010014  0.00098274 0.00082316 0.00083573 0.00076923 0.0026377
 0.00094185 0.0010014 ]


In [52]:
print(1.96*max_min_std / np.sqrt(num_iterations))

[0.00025296 0.00018244 0.00034786 0.00015057 0.00026523 0.00132488
 0.00028564 0.00025296]


In [53]:
env = np.average(env,axis=1)
po = np.average(po,axis=1)
prop = np.average(prop,axis=1)
linf = np.average(linf,axis=1)
l1 = np.average(l1, axis=1)
max_min = np.average(max_min, axis=1)

In [54]:
print('proportional, et_online, et_full, hope_online, hope_full, hope_online_v2, hope_full_v2, offline')
print('envy:')
print(env)
print(env_std* 1.96 / np.sqrt(num_iterations))
print('po')
print(np.around(po, decimals=5))
print(po_std* 1.96 / np.sqrt(num_iterations))
print('prop')
print(prop)
print(prop_std* 1.96 / np.sqrt(num_iterations))
print('sum')
print(env+po+prop)
print((env_std + po_std + prop_std)* 1.96 / np.sqrt(num_iterations))
print('linf')
print(linf)
print(linf_std* 1.96 / np.sqrt(num_iterations))
print('l1')
print(l1)
print(l1_std* 1.96 / np.sqrt(num_iterations))
print('utility')
print(max_min)
print(max_min_std* 1.96 / np.sqrt(num_iterations))

proportional, et_online, et_full, hope_online, hope_full, hope_online_v2, hope_full_v2, offline
envy:
[0.         0.00161613 0.00249126 0.00124008 0.00205496 0.01592915
 0.002903   0.        ]
[0.         0.00019623 0.00030249 0.00015057 0.00024952 0.00193414
 0.00035249 0.        ]
po
[0.      0.00071 0.00181 0.00033 0.00172 0.      0.00352 0.     ]
[1.43791707e-19 8.59388020e-05 2.19970993e-04 3.99732964e-05
 2.08954098e-04 7.18958536e-20 4.27917045e-04 1.43791707e-19]
prop
[0.         0.00107694 0.00237766 0.00081443 0.00079323 0.0050176
 0.00215838 0.        ]
[0.00000000e+00 1.30763260e-04 2.88699158e-04 9.88889529e-05
 9.63151547e-05 6.09243527e-04 2.62073297e-04 0.00000000e+00]
sum
[1.18423789e-18 3.40084541e-03 6.68055887e-03 2.38371991e-03
 4.56908837e-03 2.09467520e-02 8.58561727e-03 1.18423789e-18]
[1.43791707e-19 4.12935079e-04 8.11162159e-04 2.89434376e-04
 5.54784660e-04 2.54338191e-03 1.04247683e-03 1.43791707e-19]
linf
[0.00039507 0.00129389 0.00061907 0.00098409 0.0005

In [55]:
print(np.average(run_time, axis=1))

[3.522370e-05 1.714560e-04 8.929860e-05 4.610846e-04 3.164598e-04]
